<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Dense_BIC_Noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (dense)

In [23]:
# Function to generate CPDs with added noise
def generate_cpds_with_noise(noise_factor=0.85):
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for EI given IR
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    # Add noise to the CPDs by adjusting the probabilities slightly
    ir_probs = np.clip(ir_probs + np.random.normal(0, noise_factor, ir_probs.shape), 0, 1)
    ei_given_ir_probs = np.clip(ei_given_ir_probs + np.random.normal(0, noise_factor, ei_given_ir_probs.shape), 0, 1)
    sp_probs = np.clip(sp_probs + np.random.normal(0, noise_factor, sp_probs.shape), 0, 1)

    # Normalize after adding noise to ensure valid probability distributions
    ir_probs /= ir_probs.sum()
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, ei_given_ir_probs, sp_probs

# Function to generate and save samples with noise
def generate_and_save_samples(ir_probs, ei_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities given `IR`
        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir[ei_state_idx]

        # Sample `SP` state based on `SP` probabilities given `IR` and `EI`
        sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Sample sizes and noise factor
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]
noise_factor = 0.1  # You can adjust this value to test different levels of noise

# Generate and save samples with noise for the specified sample sizes
for size in sample_sizes:
    ir_probs, ei_given_ir_probs, sp_probs = generate_cpds_with_noise(noise_factor)
    generate_and_save_samples(ir_probs, ei_given_ir_probs, sp_probs, size, f'combined_probabilities_with_noise_{size}.csv')

print("\nGeneration and saving of individual noisy samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+============+===========+=================================================+===================+=========================+
|  0 | medium     |    0.4111 | good       |    0.3121 | 0.3669, 0.3394, 0.2938                          | increase          |                  0.2938 |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|  1 | low        |    0.4775 | average    |    0.2873 | 0.5337, 0.3227, 0.1436                          | decrease          |                  0.5337 |
+----+------------+------

# LBN Dense BIC & Entropy

In [24]:
# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the noisy dataset for the current sample size
    noisy_data_file = f'combined_probabilities_with_noise_{sample_size}.csv'
    df_dense = pd.read_csv(noisy_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR_State'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI_State'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['Chosen_SP_State'].map(sp_map)

    # Split the data into training, validation, and test sets
    X = df_dense[['IR_encoded', 'EI_encoded']]
    y = df_dense['SP_encoded']

    # Split into training (70%) and temp (30%) for validation and test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)
    print("Training data:")
    print(df_train.head())

    # Enforce the inclusion of `SP_encoded` using priors (fixed edges)
    priors = [
        ('IR_encoded', 'SP_encoded'),
        ('EI_encoded', 'SP_encoded')
    ]

    # Perform Hill Climb Search with priors (fixed edges)
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=BicScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork and print edges
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    if len(best_model.edges()) == 0:
        print("No edges learned. Skipping to next sample size.")
        continue

    # Try fitting parameters using BayesianEstimator
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=5)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Check model validity
    if not best_model.check_model():
        print("Model check failed.")
        continue

    # --- K-L Divergence and Standard Deviation Calculation Block ---
    # Predict on test data
    try:
        inference = best_model.predict(X_test)
        predicted_probabilities = inference['SP_encoded']

        # Calculate K-L divergence and standard deviation between the ground truth and learned BN
        ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions to have the same set of categories and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        # Calculate K-L divergence with smoothing
        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)

        # Standard deviation between predicted probabilities and actual probabilities
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        # Append results for this sample size
        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        # Print the K-L divergence and standard deviation for this sample size
        print(f"\nResults for sample size {sample_size}:")
        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))
        continue

# After processing all sample sizes, save results to CSV and display them
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_bic_dense_with_noise_results.csv', index=False)

# Print all results after saving to CSV
print("\nAll results have been saved to 'kl_std_bic_dense_with_noise_results.csv'.")
print(results_df)


Processing sample size: 50
Training data:
    IR_encoded  EI_encoded  SP_encoded
6            2           1           2
41           1           0           0
46           0           2           0
47           0           0           1
15           1           2           1


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/4 [00:00<?, ?it/s]


Results for sample size 50:
K-L Divergence: 2.7505
Standard Deviation: 0.2700

Processing sample size: 100
Training data:
    IR_encoded  EI_encoded  SP_encoded
11           1           0           2
47           2           0           1
85           1           0           2
28           1           1           2
93           1           1           2


...


Results for sample size 19000:
K-L Divergence: 0.3714
Standard Deviation: 0.2304

Processing sample size: 20000
Training data:
       IR_encoded  EI_encoded  SP_encoded
17218           1           1           2
15188           0           0           2
11295           1           2           1
19772           1           1           2
13072           2           0           0


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded'), ('EI_encoded', 'IR_encoded')]
Model fitted successfully.


  0%|          | 0/9 [00:00<?, ?it/s]


Results for sample size 20000:
K-L Divergence: 0.0331
Standard Deviation: 0.0903

All results have been saved to 'kl_std_bic_dense_with_noise_results.csv'.
    Sample_Size  K-L_Divergence  Standard_Deviation
0            50        2.750473            0.270031
1           100        5.618299            0.217732
2           150        0.066422            0.127997
3           200        7.789262            0.259629
4           250        0.073049            0.134185
5           300        0.024027            0.065420
6           350        0.401136            0.262800
7           400        0.185696            0.209497
8           450        0.056467            0.098284
9           500        2.689326            0.121228
10          550        0.050420            0.111730
11          600        1.652700            0.211403
12          650        0.312375            0.108311
13          700        5.787073            0.218149
14          750        4.605243            0.188283
15         